# <font style="color:blue">Project 2: Kaggle Competition - Classification</font>

#### Maximum Points: 100

<div>
    <table>
        <tr><td><h3>Sr. no.</h3></td> <td><h3>Section</h3></td> <td><h3>Points</h3></td> </tr>
        <tr><td><h3>1</h3></td> <td><h3>Data Loader</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>2</h3></td> <td><h3>Configuration</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>3</h3></td> <td><h3>Evaluation Metric</h3></td> <td><h3>10</h3></td> </tr>
        <tr><td><h3>4</h3></td> <td><h3>Train and Validation</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>5</h3></td> <td><h3>Model</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>6</h3></td> <td><h3>Utils</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>7</h3></td> <td><h3>Experiment</h3></td><td><h3>5</h3></td> </tr>
        <tr><td><h3>8</h3></td> <td><h3>TensorBoard Dev Scalars Log Link</h3></td> <td><h3>5</h3></td> </tr>
        <tr><td><h3>9</h3></td> <td><h3>Kaggle Profile Link</h3></td> <td><h3>50</h3></td> </tr>
    </table>
</div>


## <font style="color:green">1. Data Loader [10 Points]</font>

In this section, you have to write a class or methods, which will be used to get training and validation data loader.

You need to write a custom dataset class to load data.

**Note; There is   no separate validation data. , You will thus have to create your own validation set, by dividing the train data into train and validation data. Usually, we do 80:20 ratio for train and validation, respectively.**


For example:

```python
class KenyanFood13Dataset(Dataset):
    """
    
    """
    
    def __init__(self, *args):
    ....
    ...
    
    def __getitem__(self, idx):
    ...
    ...
    
    
```

```
def get_data(args1, *agrs):
    ....
    ....
    return train_loader, test_loader
```

## <font style="color:green">Class_counts: (very uneven)</font>
* id: 0 'githeri': 479,
* id: 1 'ugali': 628,
* id: 2 'kachumbari': 494,
* id: 3 'matoke': 483,
* id: 4 'sukumawiki': 402,
* id: 5 'bhaji': 632,
* id: 6 'mandazi': 620,
* id: 7 'kukuchoma': 173,
* id: 8 'nyamachoma': 784,
* id: 9 'pilau': 329,
* id:10 'chapati': 862,
* id:11 'masalachips': 438,
* id:13 'mukimo': 212

Name: class, dtype: int64, sum: 6536



In [1]:
%matplotlib inline

In [2]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 1.6 MB/s eta 0:00:0000:0100:01


In [3]:
# Import neccesaary libraries
import os
import time
import collections
from dataclasses import dataclass
import sys

# third party library
import timm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image


# Pytorch related
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import WeightedRandomSampler
from torch.utils.data import DataLoader
from torch.utils.data import Subset
from torch.optim import SGD  
from torch.optim import Adam
from torch.optim import AdamW
from torch.utils.tensorboard import SummaryWriter
from torchvision.io import read_image
from torchvision import transforms
from torchvision.models import resnet152, resnet18
from torchvision.transforms import AutoAugmentPolicy, AutoAugment

In [4]:
def small_image_preprocess_transforms():
    """pre_process for KenyanFood13Testset holding original database(No need to hold big image)"""
    small_image_preprocess = transforms.Compose([
        transforms.Resize(1),
        transforms.ToTensor()
    ])
    
    return small_image_preprocess


### from here to below will change to Tensor
def image_preprocess_transforms():
    """pre_process for KenyanFood13Testset """
    image_preprocess = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
        transforms.Resize((224,224)),
        transforms.ToTensor()
    ])
    
    return image_preprocess


def train_preprocess():
    """resize_preprocess() + couple transformation to improve accuracy in training, ToTensor and Normalization"""
    transforms_train = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
        transforms.Resize((224,224)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        # Somehow this doesn't work
        #transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio = (0.33, 0.33), value= 0, inplace = False), 
        transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5),
        transforms.RandomRotation(degrees=10),
        transforms.RandomPosterize(bits=2),
        transforms.RandomChoice([
            AutoAugment(AutoAugmentPolicy.CIFAR10),
            AutoAugment(AutoAugmentPolicy.IMAGENET),
            transforms.TrivialAugmentWide()
            ]),
        transforms.ToTensor(),
        transforms.Normalize(tc.mean, tc.std)        
    ])
    return transforms_train
    
    
def validation_preprocess():
    validation_train = transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(tc.mean, tc.std)
    ])
    return validation_train


In [5]:
# Datasets Classes
class KenyanFood13Dataset(Dataset):
    """create KenyanFood Dataset from annotations_file and images"""
    def __init__(self, transform = None):
        self.img_labels = pd.read_csv(tc.annotation_file)
        self.img_dir = tc.img_dir
        # Make list of the classes list will give index (iterable) -> 13 classes
        self.classes = list(self.img_labels['class'].unique()) 
        # Make dictionary set of the classes 
        self.dict_classes = dict(enumerate(self.img_labels['class'].unique()))
        self.transform = transform
        
        # weigths of each class normalized (total = 1.0)
        def _inverse_ratio():
            """calculate inverse ratio of number of each classes for weighted sampler usage"""
            series = self.img_labels['class'].value_counts()
            total = series.sum()
            for index, _ in enumerate(series):
                series.iloc[index] = series.iloc[index]/total
            return series
        
        # This inner method won't work for inheritance since it is inside __init__()
        def _calc_sample_weights():
            class_counts = self.img_labels['class'].value_counts()
            sample_weights = [1/class_counts[i] for i in self.img_labels['class']]
            return sample_weights
        
        # sample_weights
        self.samples_weights = [self.img_labels['class'].value_counts()[i] for i in self.img_labels['class']]       
    
    # need this method to get original dataset image_labels from child class
    def _get_img_labels(self):
        return self.img_labels  
        
    def __len__(self):
        return len(self.img_labels)
    
    def __getitem__(self, index):
        """Return (image, target) after resizing and preprocessing 
        iloc[index, 0] will return ids (e.g.,14278962987112149800) of each row"""
        img_path = os.path.join(self.img_dir, str(self.img_labels.iloc[index, 0])+".jpg")     
        image = Image.open(img_path)

        # label is string so will return index (pytorch cannot make string to tensor)
        # iloc[index, 1] will return the class_name (e.g., githeri) for id in [index, 1] (e.g., 14278962987112149800)
        label_index = self.class2index(self.img_labels.iloc[index, 1])  # returns int

        if self.transform:
            image = self.transform(image)

        return image, label_index #image: Image or Tensor ,  label_index: int -> Does this needs to be Tensor?
    
    
    def class2index(self, class_name:str)-> int:
        """Returns the index of a given class."""
        return self.classes.index(class_name)
    
    
    def index2class(self, class_index:int)-> str:
        """Returns the class of a given index."""
        return self.classes[class_index]


class KenyanFood13SplitDataset(KenyanFood13Dataset):
    def __init__(self, transform = None, train= True):
        # above __init__() will override super class init(), so needs to call super()__init__ for initialization
        super().__init__(transform = transform)
        #self.img_labels = None
#       print(f"self.img_labels in __init()__ first: {self.img_labels}\n") # 6536

        # need this method to get original dataset image_labels
        image_labels = super()._get_img_labels()

        # Overrides self.img_labels by ratio of split
        if train:
            self.img_labels = image_labels.iloc[:int(len(image_labels) * tc.train_split)]
        else: #for validation
            self.img_labels = image_labels.iloc[int(len(image_labels) * tc.train_split):]
        
        # Override the sample_weights since train database has splitted database, needed for weighted Random Sampler
        self.samples_weights = [self.img_labels['class'].value_counts()[i] for i in self.img_labels['class']]
    
class KenyanFood13Testset(Dataset):
    """Kenyan food test dataset, contains original KenanFood13Dataset for image2class and class2image methods"""
    def __init__(self, transform = image_preprocess_transforms()):
        self.img_dir = tc.img_dir
        self.test_labels = pd.read_csv(tc.test_csv_file)
        self.transform = transform
        # this is to use class2index() and class2index()
        self.base_dataset = KenyanFood13Dataset(small_image_preprocess_transforms())

    def __getitem__(self, index):
        """Retrieves one item from the dataset."""
        
        # get img path from test_labels. img_dir + id in csv file (per index) + ".jpg" 
        img = os.path.join(self.img_dir, str(self.test_labels.iloc[index, 0]) + '.jpg')
        
        image = Image.open(img)

        if self.transform:
            image = self.transform(image)

        return image
    
    def __len__(self):
        return len(self.test_labels)

## <font style="color:green">2. Configuration [5 Points]</font>

**Define your configuration here.**

For example:


```python
@dataclass
class TrainingConfiguration:
    '''
    Describes configuration of the training process
    '''
    batch_size: int = 10 
    epochs_count: int = 50  
    init_learning_rate: float = 0.1  # initial learning rate for lr scheduler
    log_interval: int = 5  
    test_interval: int = 1  
    data_root: str = "/kaggle/input/pytorch-opencv-course-classification/" 
    num_workers: int = 2  
    device: str = 'cuda'  
    
```

In [6]:
# Settings and configurations
@dataclass
class SystemConfiguration:
    seed:int = 42
        
# Specifiy all the data needed in dataclass named TrainingConfiguration
@dataclass
class TrainingConfiguration:
    batch_size: int = 13
    epoch_count: int = 20
    init_learning_rate: float = 0.0001
    lr: float =0.00001
    log_interval: int = 5
    test_interval: int = 1
    data_root: str = "/kaggle/input/opencv-pytorch-dl-course-classification/"
    model_dir: str = '/kaggle/working/models/'
    log_dir: str = '/kaggle/working/logs'
    submission_csv: str = '/kaggle/working/submission.csv'
#     submission_csv: str = '/content/submission.csv'
#     data_root: str = "/content/"    
#     model_dir: str = '/content/models/'
#     log_dir: str = '/content/logs'
    annotation_file: str = data_root + "train.csv"
    test_csv_file: str = data_root + "test.csv"
    img_dir: str = data_root + "images/images/"

    num_workers: int = 2
    device: str = 'cpu'
    train_split: float = 0.8
    random_seed: int = 42
    model_name: str = "resnet152"
    res18_layers :tuple = (2,2, 2,2)
    res34_layers :tuple = (3,4, 6,3)
    res50_layers :tuple = (3,4, 6,3)
    res101_layers:tuple = (3,4,23,3)
    res152_layers:tuple = (3,8,36,3)

    classes: tuple = ('githeri', 'ugali', 'kachumbari', 'matoke', 'sukumawiki', 'bhaji', 'mandazi', 'kukuchoma', 'nyamachoma', 'pilau', 'chapati', 'masalachips', 'mukimo')
    dataset_len: int = 6536
    #below is calculated from KenyanFood13 pictures
    #mean: torch.tensor = torch.tensor([0.5768, 0.4622, 0.3460])
    #std: torch.tensor = torch.tensor([0.2699, 0.2739, 0.2826])
    
    #below mean and standard are for Resnet18 https://pytorch.org/hub/pytorch_vision_resnet/
    mean: torch.tensor = torch.tensor([0.485, 0.456, 0.406])
    std: torch.tensor  = torch.tensor([0.229, 0.224, 0.225])
        
    tb_writer: SummaryWriter = SummaryWriter(log_dir)
    criterion: torch.nn.CrossEntropyLoss = torch.nn.CrossEntropyLoss()        

def setup_system(SystemConfiguration):
    torch.manual_seed(system_config.seed)
    if torch.cuda.is_available():
        tc.device = torch.device("cuda:0")
        torch.backends.cudnn_benchmark_enabled = True
        torch.backends.cudnn.deterministic = True
        
        
tc = TrainingConfiguration()
system_config = SystemConfiguration()
setup_system(system_config)

for path in [tc.log_dir, tc.model_dir]:
    if not os.path.exists(path):
        os.makedirs(path)

## <font style="color:green">3. Evaluation Metric [10 Points]</font>

**Define methods or classes that will be used in model evaluation. For example, accuracy, f1-score etc.**

## <font style="color:green">4. Train and Validation [5 Points]</font>


**Write the methods or classes to be used for training and validation.**

In [7]:
def train(model, train_loader, optimizer) -> float:
    """train by model by given train_loader(and its datasets) and optimizer, return average loss and accuracy"""
    total_loss = 0.0
    total_acc = 0
    count = 0
    model.to(tc.device)
    model.train()
    
    for inputs, labels in tqdm(train_loader):
        count += len(labels)
        # move to target device (GPU or CPU)
        inputs = inputs.to(tc.device)
        labels = labels.to(tc.device)
        
        # prediction
        outputs = model(inputs)
        # calculate loss
        loss = tc.criterion(outputs, labels)
        # initialize the gradients -> calucalte the gradients -> Update the gradients
        optimizer.zero_grad()     
        loss.backward()
        optimizer.step()
        
        # adds up loss pf number of batches
        total_loss += loss.item()
        # calculates accumulative average loss by dividing total loss by length of dataset predicted so far
        avg_loss = total_loss / count
        
        # get highest predicted value
        predicted = torch.max(outputs, axis=1)[1]
        # calculates accumulative average accuracy by dividing total loss by length of dataset predicted so far
        total_acc  += (predicted == labels).sum().item()
        avg_acc =  total_acc / count
        
    return avg_loss, avg_acc

In [8]:
def validate(model, validation_loader):
    """validate by model by given validation_loader(and its datasets) , return average loss"""
    total_loss = 0.0
    total_acc = 0
    count = 0
    model.to(tc.device)
    model.eval()
    
    with torch.no_grad():
        for inputs, labels in tqdm(validation_loader):
            count += len(labels)
            # move to target device (GPU or CPU)
            inputs = inputs.to(tc.device)
            labels = labels.to(tc.device)
            
            # prediction
            outputs = model(inputs)
            # calculate loss
            loss = tc.criterion(outputs, labels)
            
            # adds up loss pf number of batches
            total_loss += loss.item()
            # calculates accumulative average loss by dividing total loss by length of dataset predicted so far
            avg_loss = total_loss / count
            
            # get highest predicted value
            predicted = torch.max(outputs, axis=1)[1]
            # calculate number of correct answer
            total_acc  += (predicted == labels).sum().item()
            # calculates accumulative average accuracy by dividing total loss by length of dataset predicted so far
            avg_acc =  total_acc / count
        
            
    return avg_loss, avg_acc
        

In [9]:
def predict(trained_model, prediction_loader) -> list:
    """prediction with model and test_loader, returns list of prediction result, assume to be used for inference ONLY"""  
    predictions = torch.tensor([]).to(tc.device)
    for inputs in prediction_loader:
        inputs = inputs.to(tc.device)
        outputs = trained_model(inputs)
        # get the predicted result by index number, each prediction will be done by tc.batch_size so predicted is array of batch_size
        predictions = torch.hstack((predictions, torch.max(outputs, axis=1)[1]))
        
    return predictions.tolist()


## <font style="color:green">5. Model [5 Points]</font>

**Define your model in this section.**

**You are allowed to use any pre-trained model.**

In [10]:
#implement Resnet from Scratch
# conv3x3 and conv1x1 as static method since used a lot.
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride=stride, padding=1, bias=False)

def conv1x1(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)


In [11]:
class BasicBlock(nn.Module):
    # at the last layer of each block, the channel is x1, for example resnet30 first Bottleneck,
    # 1x1 conv 64 -> 3x3 conv 64
    expansion = 1

    def __init__(self, in_channels, base_channels, stride=1):
        """
        Constructor

        Parameters
        ----------
        in_channels : int
            number of channels for input
        base_channels : int
            number of channels used in this basic block
        stride : int
            stride used in this basic block
        """
        super().__init__()
        self.conv1 = conv3x3(in_channels, base_channels, stride)
        self.bn1 = nn.BatchNorm2d(base_channels)
        self.relu = nn.ReLU(inplace=True)  # inplace=True will directory change the value
        self.conv2 = conv3x3(base_channels, base_channels)
        self.bn2 = nn.BatchNorm2d(base_channels)

        # First residual block's number of input channel and output channel is different (e.g., conv3_x in res18 has input channel as 64 but out put is 128)
        # And also the size (56 x 56 -> 28 x 28),
        if in_channels != base_channels * self.expansion:
            self.shortcut = nn.Sequential(
                conv1x1(in_channels, base_channels * self.expansion, stride=2),  # stride = 2 due to make the size half
                nn.BatchNorm2d(base_channels * self.expansion)
            )
            print(f"self.shortcut for in_channels != channels * self.expansion: {self.shortcut}")
        else:
            self.shortcut = nn.Sequential()
            print(f"self.shortcut for in_channels == channels * self.expansion: {self.shortcut}")

        print(
            f"##### in_channels:{in_channels}  base_channels:{base_channels} conv1 stride: {stride}, self.shortcut {self.shortcut}")

    def forward(self, x):
        """

        Parameters
        ----------
        self
        x

        Returns
        -------

        """
        out = self.conv1(x)
        print(f"conv1(x).size(): {out.shape}")
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += self.shortcut(x)
        print(f"self.shortcut(x).shape: {self.shortcut(x).shape}")
        print(f"out += self.shortcut(x): {out.shape}")
        out = self.relu(out)

        return out

In [12]:
class Bottleneck(nn.Module):
    # at the last layer of each block, the channel is x4, for example resnet50 first Bottleneck,
    # 1x1 conv 64 -> 3x3 conv 64 -> 1x1 conv 256
    expansion = 4

    def __init__(self, in_channels, base_channels, stride=1):
        """

        Parameters
        ----------
        in_channels
        base_channels
        stride
        """
        super().__init__()
        self.conv1 = conv1x1(in_channels, base_channels)
        self.bn1 = nn.BatchNorm2d(base_channels)
        self.conv2 = conv3x3(base_channels, base_channels, stride)
        self.bn2 = nn.BatchNorm2d(base_channels)
        # third one has 4x more channels as out put
        self.conv3 = conv1x1(base_channels, base_channels * self.expansion)
        self.bn3 = nn.BatchNorm2d(base_channels * self.expansion)
        self.relu = nn.ReLU(inplace=True)

        if in_channels != base_channels * self.expansion:
            self.shortcut = nn.Sequential(
                # need stride as parameter since conv2_x is stride=1, and others are 2
                conv1x1(in_channels, base_channels * self.expansion, stride),
                nn.BatchNorm2d(base_channels * self.expansion),
            )
            print(f"self.shortcut for in_channels != channels * self.expansion: {self.shortcut}")
        else:
            self.shortcut = nn.Sequential()
            print(f"self.shortcut for in_channels == channels * self.expansion: {self.shortcut}")

        print(
            f"##### in_channels:{in_channels}  base_channels:{base_channels} conv2 stride: {stride}, self.shortcut {self.shortcut}")

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        out += self.shortcut(x)
        print(f"self.shortcut(x).shape: {self.shortcut(x).shape}")
        print(f"out += self.shortcut(x): {out.shape}")

        out = self.relu(out)

        return out

In [13]:

class Resnet(nn.Module):
    def __init__(self, block_type, layers: tuple, num_classes=1000):
        """

        Parameters
        ----------
        block_type: BasicBlock, Bottleneck
            Whether this resnet use BasicBlock (Res18/Res34) or Bottleneck(Res50 or higher)
        layers: tuple
            contains number of covolution in each layer, for example for Resnet50 (3,4,6,4)
        num_classes: int
            number of output classes
        """
        super().__init__()

        self.in_channels = 64  # initial channel for all type is 64

        # initial conv1_x layer, number of input_channel = 3(R,G,B), number of output_channel = 64
        # Input image size 224*224*3(RGB), padding = (3,3) to make size 230 * 230 ->  112*112 with stride (2,2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3),
                               bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)

        # conv2_x layer, input size is (batch_size, 64, 112, 112)
        self.maxpool = nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.conv2_x = self._create_layer(block_type, 64, layers[0], stride=1)  # for resnet50 layers[0] = 3
        self.conv3_x = self._create_layer(block_type, 128, layers[1], stride=2)  # for resnet50 layers[1] = 4
        self.conv4_x = self._create_layer(block_type, 256, layers[2], stride=2)  # for resnet50 layers[2] = 6
        self.conv5_x = self._create_layer(block_type, 512, layers[3], stride=2)  # for resnet50 layers[3] = 3
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block_type.expansion, num_classes)

    def _create_layer(self, block_type, base_channels, repeated_conv_blocks, stride):
        """
        create layers, conv2_x, conv3_x, conv4_x and conv5_x in Resnet18, 34, 50, 101 and 152

        Parameters
        ----------
        block_type:BasicBlock, Bottleneck
            Whether this resnet use BasicBlock (Res18/Res34) or Bottleneck(Res50 or higher)
        base_channels: int
            for conv2_x: 64, conv3_x:128 , conv4_x = 256, conv5_x 512. These numbers are common across all Resnet type
        repeated_conv_blocks : int
            number of convolution blocks repeated, for example res50 conv2_x repeats 3 times and conv4_x repeats 6 times
        stride: int or tuple(int,int)

        Returns
        -------
            nn.Sequential
        """
        layers = []

        # first Residual basic block (or bottleneck), constructs basic block (or bottleneck) and appends to list
        layers.append(block_type(self.in_channels, base_channels, stride))
        # rest of Residual basic block
        self.in_channels = base_channels * block_type.expansion
        for _ in range(1, repeated_conv_blocks):
            layers.append(block_type(self.in_channels, base_channels))

        return nn.Sequential(*layers)

    def forward(self, x):

        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)

        x = self.maxpool(x)

        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [14]:
# Use pretrained one so comment out
# def resnet18():
#     return ResNet(BasicBlock, tc.res18_layers)
# def resnet34():
#     return ResNet(BasicBlock, tc.res34_layers)
# def resnet50():
#     return ResNet(Bottleneck, tc.res50_layers)
# def resnet101():
#     return Resnet(Bottleneck, tc.res101_layers)
# def resnet152():
#     return Resnet(Bottleneck, tc.re152_layers)

In [15]:
# # Resnet 18 (or 152) and change last layer output to 13
# model = resnet152(pretrained = True)
# # Change the last output to 13
# fc_in_features = model.fc.in_features
# model.fc = torch.nn.Linear(fc_in_features, (len(tc.classes)))
# model = model.to(tc.device)


In [16]:
class AdditionalDenseLayer(nn.Module):
    def __init__(self):
        super().__init__()
        self.classifier_layer = nn.Sequential(
            #nn.Linear(1280, 1024), # efficient net
            nn.Linear(2048, 1024), # resnetrest 
            nn.BatchNorm1d(1024),
            nn.Dropout(0.2),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.BatchNorm1d(512),   
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, len(tc.classes))
        )
    
    def forward(self, x):
        x = self.classifier_layer(x)
        
        return x

## <font style="color:green">6. Utils [5 Points]</font>

**Define those methods or classes, which have  not been covered in the above sections.**

In [17]:
## util functions

def get_mean_std(dataset):
    """returns mean and standard deviation of dataset given, since this method will take long time, once calcuated, not been used"""
    # calculated mean: tensor([0.5768, 0.4622, 0.3460]), std:tensor([0.2699, 0.2739, 0.2826]):
    loader = DataLoader(dataset)
    
    batch_mean = torch.zeros(3) # tensor([0,0,0])
    batch_mean_sqrd = torch.zeros(3)
    
    for batch_data, _ in loader:
        batch_mean += batch_data.mean(dim=(0,2,3))
        batch_mean_sqrd += (batch_data **2).mean(dim=(0,2,3))
        
    mean = batch_mean / len(loader)
    var = (batch_mean_sqrd) / len(loader) - (mean **2)
    
    std = var ** .5
    
    print("mean: {}, std:{}:".format(mean, std))
    return mean, std


def eval_loss(data_loader, device, model, criterion):
    """evaluate losses and can use this for visualize"""
    for images, labales in data_loader:
        break
        
    inputs = images.to(device)
    labels = labels.to(device)
    
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    return loss

def evaluate_history(history):
    import matplotlib.pyplot as plt
    print("initial stats: Loss{:.5f}  Accuracy{:.5f}".format(history[0,3], history[0,4]))
    print("final stats: Loss{:.5f}  Accuracy{:.5f}".format(history[-1,3], history[-1,4]))
    
    num_epochs = len(history)
    unit = num_epochs/10
    
    # Display learning curve (Loss)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,1], 'b', label='train')
    plt.plot(history[:,0], history[:,3], 'k', label='validation')
    plt.xticks(np.arange(0,num_epochs+1, unit))
    plt.xlabel('# of iteration')
    plt.ylabel('loss')
    plt.title('learning curve (Loss)')
    plt.legend()
    plt.show()

    # Display learning curve (Accuracy)
    plt.figure(figsize=(9,8))
    plt.plot(history[:,0], history[:,2], 'b', label='train')
    plt.plot(history[:,0], history[:,4], 'k', label='validation')
    plt.xticks(np.arange(0,num_epochs+1,unit))
    plt.xlabel('# of iteration')
    plt.ylabel('loss')
    plt.title('learning curve (Accuracy')
    plt.legend()
    plt.show()


# model save and load functions
def save_model(model, device):
    if not os.path.exists(tc.model_dir):
        os.makedirs(tc.model_dir)
    
    model_path = os.path.join(tc.model_dir, tc.model_name)
    
    if device == "cuda":
        model.to("cpu")
    
    #torch.save(model.state_dict(), model_path + "best_model.pt")
    torch.save(model.state_dict(), os.path.join(tc.model_dir, "best_model.pt"))
    
    if device == "cuda":
        model.to("cuda")
    
    return

def load_model(model):
    
    #model_path = os.path.join(tc.model_dir, tc.model_name)
    model.load_state_dict(torch.load(tc.model_dir + 'best_model.pt'))
    
    return model

* id: 0 'githeri': 479,
* id: 1 'ugali': 628,
* id: 2 'kachumbari': 494,
* id: 3 'matoke': 483,
* id: 4 'sukumawiki': 402,
* id: 5 'bhaji': 632,
* id: 6 'mandazi': 620,
* id: 7 'kukuchoma': 173,
* id: 8 'nyamachoma': 784,
* id: 9 'pilau': 329,
* id:10 'chapati': 862,
* id:11 'masalachips': 438,
* id:13 'mukimo': 212

In [18]:
#tools to use matplot to visualize data balanced each batch
def visualize_batch_data(dataloader, with_visual_chart = True , with_class_string = True) -> dict:
    """visualize what class is in the batch to visually see the equalness of the samples"""
    #{0: 'githeri', 1: 'ugali', 2: 'kachumbari', 3: 'matoke', 4: 'sukumawiki', 5: 'bhaji', 6: 'mandazi', 7: 'kukuchoma', 8: 'nyamachoma', 9: 'pilau', 10: 'chapati', 11: 'masalachips', 12: 'mukimo'}
    # total_number_appeared: Counter({10: 862, 8: 784, 5: 632, 1: 628, 6: 620, 2: 494, 3: 483, 0: 479, 11: 438, 4: 402, 9: 329, 12: 212, 7: 173})
    total_num_images = len(data_loader.dataset)
    total_number_appeared = collections.Counter()
    number_appeared_per_batch = []
    dict_classes = data_loader.dataset.dict_classes
    
    for index, (_, label_index) in enumerate(data_loader): # don't use image
#         class_ids, class_counts = torch.unique(label_index, return_counts=True) # return in torch.tensor
        number_appeared = collections.Counter(label_index.tolist())
        print(f"batch:{index} number_appeared:{number_appeared}")
        
        # add zero for sample which was not picked, and add that label_index:0 to Counter
        while len(number_appeared) < len(dict_classes):
            for i in range(len(number_appeared)):
                number_appeared.setdefault(i,0)
            print(f"batch:{index} number_appeared_after_setdefault:{number_appeared}")
        # for total count
        total_number_appeared += number_appeared

            
        # sort based on keys (label_index) and cast to dict for visualization
        dict_number_appeared = dict(sorted(number_appeared.items()))
        #print(f"dict_number_appeared:{dict_number_appeared}")
        number_appeared_per_batch.append(dict_number_appeared)
        
    if with_visual_chart:
        """create bar graph for all batches which class had samples"""
        fig, axes = plt.subplots(int(tc.batch_size/4), 4 , figsize = (20, 20))
        for i in range(int(tc.batch_size/4)):
            for j in range(4):
                axes[i][j].bar(number_appeared_per_batch[int(j+(i*4))].keys(), list(number_appeared_per_batch[int(j+(i*4))].values()))
                if with_class_string:
                    axes[i][j].set_xticks(list(range(len(dict_classes))))
                    axes[i][j].set_xticklabels(list(dict_classes.values()))
                else:
                    axes[i][j].set_xticks(list(range(len(dict_classes))))
                    axes[i][j].set_xticklabels(list(dict_classes.keys()))

    
    # return dict type, key=class name, value=how many times appeared
    if with_class_string:
        return {data_loader.dataset.index2class(key):value for key, value in total_number_appeared.items()}
    
    # return dict type, key=class_id, value=how many times appeared
    return dict(total_number_appeared)


## <font style="color:green">7. Experiment [5 Points]</font>

**Choose your optimizer and LR-scheduler and use the above methods and classes to train your model.**

In [19]:
# use timm.resnetrs50 as backborn and add couple linear layers
model = timm.create_model('resnetrs50', pretrained=True, num_classes=1000) 

#model = timm.create_model('efficientnet_b0', pretrained=True, num_classes=1000) 

# freeze the backborn and do training (resnet)
for param in model.parameters():
    param.requires_grad = False
model.fc = AdditionalDenseLayer()

# # freeze the backborn and do training (effcient net)
# for param in model.parameters():
#     param.requires_grad = False
# model.classifier = AdditionalDenseLayer()

# model

Downloading: "https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rs-weights/resnetrs50_ema-6b53758b.pth" to /root/.cache/torch/hub/checkpoints/resnetrs50_ema-6b53758b.pth


In [20]:
train_dataset = KenyanFood13SplitDataset(transform = train_preprocess(),train = True)
validation_dataset = KenyanFood13SplitDataset(transform = validation_preprocess(), train = False)
sequential_sampler = torch.utils.data.SequentialSampler(train_dataset)
weight_sampler = WeightedRandomSampler(weights=train_dataset.samples_weights, num_samples=len(train_dataset), replacement=True)
train_loader = DataLoader(train_dataset, sampler = weight_sampler, batch_size = tc.batch_size, shuffle = False)
validation_loader = DataLoader(validation_dataset, batch_size = tc.batch_size, shuffle = False )
# create optimizer
optimizer = AdamW(model.parameters(), lr=tc.init_learning_rate)
print(f"length of train_dataset {len(train_dataset.samples_weights)}")

length of train_dataset 5228


In [21]:
best_val_loss = np.Inf
history = np.zeros((0,6))
for epoch in range(tc.epoch_count):
    # time one cycle of train and validate
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, validation_loader)
    elapsed_time = time.time() - start_time
    
    # save the weight when the loss gets smaller than last one
    if val_loss < best_val_loss:
        save_model(model, tc.device)
        print("WEIGHTS-ARE-SAVED")
        best_val_loss = val_loss
        
    print(f"Epoch{epoch+1}/{tc.epoch_count}, loss:{train_loss:.5f}, acc:{train_acc:.5f}, val_loss:{val_loss:.5f}, val_acc:{val_acc:.5f}, elapsed_time:{elapsed_time:.5f}")
    # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
    item = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc, elapsed_time])
    # store history for matplotlib visualization
    history = np.vstack((history, item))

    # For tensorboard
    tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_loss, epoch+1)
    tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_acc, epoch+1)
    tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch1/20, loss:0.13739, acc:0.44549, val_loss:0.11938, val_acc:0.51835, elapsed_time:224.41029


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch2/20, loss:0.11050, acc:0.53826, val_loss:0.11336, val_acc:0.54128, elapsed_time:201.91445


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch3/20, loss:0.10602, acc:0.56427, val_loss:0.10183, val_acc:0.59633, elapsed_time:197.54165


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch4/20, loss:0.10136, acc:0.57364, val_loss:0.09845, val_acc:0.60321, elapsed_time:197.71808


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch5/20, loss:0.09865, acc:0.59220, val_loss:0.10109, val_acc:0.59862, elapsed_time:197.41717


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch6/20, loss:0.09911, acc:0.58550, val_loss:0.09817, val_acc:0.59939, elapsed_time:197.32309


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch7/20, loss:0.09472, acc:0.60214, val_loss:0.09605, val_acc:0.61315, elapsed_time:199.92323


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch8/20, loss:0.09477, acc:0.59296, val_loss:0.09545, val_acc:0.61697, elapsed_time:197.98522


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch9/20, loss:0.09174, acc:0.62127, val_loss:0.09723, val_acc:0.61774, elapsed_time:201.31180


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch10/20, loss:0.09122, acc:0.61591, val_loss:0.09204, val_acc:0.63073, elapsed_time:201.82182


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch11/20, loss:0.09002, acc:0.61725, val_loss:0.09391, val_acc:0.63609, elapsed_time:202.93696


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch12/20, loss:0.08590, acc:0.63045, val_loss:0.09298, val_acc:0.61927, elapsed_time:201.16578


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch13/20, loss:0.08820, acc:0.62663, val_loss:0.09840, val_acc:0.60398, elapsed_time:202.29472


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch14/20, loss:0.08706, acc:0.63103, val_loss:0.09199, val_acc:0.62997, elapsed_time:199.27276


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch15/20, loss:0.08834, acc:0.62490, val_loss:0.08972, val_acc:0.63379, elapsed_time:201.54505


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch16/20, loss:0.08497, acc:0.63906, val_loss:0.09232, val_acc:0.62232, elapsed_time:200.35316


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch17/20, loss:0.08489, acc:0.63160, val_loss:0.09340, val_acc:0.62844, elapsed_time:202.53273


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch18/20, loss:0.08175, acc:0.65742, val_loss:0.09291, val_acc:0.63838, elapsed_time:205.00834


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch19/20, loss:0.08353, acc:0.64499, val_loss:0.09328, val_acc:0.63914, elapsed_time:203.52960


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch20/20, loss:0.08169, acc:0.65245, val_loss:0.09120, val_acc:0.64908, elapsed_time:204.56002


initial stats: Loss0.11938  Accuracy0.51835
final stats: Loss0.09120  Accuracy0.64908


In [22]:
evaluate_history(history)

In [23]:
# Unfreeze the backborn and do training again
for param in model.layer4.parameters():
    param.requires_grad = True
optimizer = AdamW(model.parameters(), lr=tc.lr)

In [24]:
# and do training again
for epoch in range(tc.epoch_count):
    # time one cycle of train and validate
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, validation_loader)
    elapsed_time = time.time() - start_time
    
    # save the weight when the loss gets smaller than last one
    if val_loss < best_val_loss:
        save_model(model, tc.device)
        print("WEIGHTS-ARE-SAVED")
        best_val_loss = val_loss
        
    print(f"Epoch{epoch+1}/{tc.epoch_count}, loss:{train_loss:.5f}, acc:{train_acc:.5f}, val_loss:{val_loss:.5f}, val_acc:{val_acc:.5f}, elapsed_time:{elapsed_time:.5f}")
    # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
    item = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc, elapsed_time])
    # store history for matplotlib visualization
    history = np.vstack((history, item))

    # For tensorboard
    tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_loss, epoch+1)
    tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_acc, epoch+1)
    tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch1/20, loss:0.08146, acc:0.65780, val_loss:0.08772, val_acc:0.66208, elapsed_time:210.04753


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch2/20, loss:0.07882, acc:0.66947, val_loss:0.08729, val_acc:0.66131, elapsed_time:210.54844


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch3/20, loss:0.07504, acc:0.67712, val_loss:0.08633, val_acc:0.67125, elapsed_time:211.62573


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch4/20, loss:0.07466, acc:0.68344, val_loss:0.08545, val_acc:0.67049, elapsed_time:209.87371


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch5/20, loss:0.07130, acc:0.70199, val_loss:0.08422, val_acc:0.67049, elapsed_time:210.15116


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch6/20, loss:0.07255, acc:0.68994, val_loss:0.08177, val_acc:0.67661, elapsed_time:208.10266


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch7/20, loss:0.07194, acc:0.70562, val_loss:0.08323, val_acc:0.66896, elapsed_time:209.22719


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch8/20, loss:0.07063, acc:0.69759, val_loss:0.08212, val_acc:0.68119, elapsed_time:209.70528


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch9/20, loss:0.06834, acc:0.71270, val_loss:0.08330, val_acc:0.67813, elapsed_time:206.78501


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch10/20, loss:0.07043, acc:0.70715, val_loss:0.08284, val_acc:0.67813, elapsed_time:207.11018


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch11/20, loss:0.06846, acc:0.71232, val_loss:0.08200, val_acc:0.69113, elapsed_time:207.81049


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch12/20, loss:0.06864, acc:0.71079, val_loss:0.08429, val_acc:0.67737, elapsed_time:207.14468


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch13/20, loss:0.06451, acc:0.72456, val_loss:0.08268, val_acc:0.68119, elapsed_time:208.78068


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch14/20, loss:0.06638, acc:0.71347, val_loss:0.08191, val_acc:0.68731, elapsed_time:208.43905


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch15/20, loss:0.06339, acc:0.72800, val_loss:0.07968, val_acc:0.68884, elapsed_time:208.85729


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch16/20, loss:0.06508, acc:0.72571, val_loss:0.08044, val_acc:0.70413, elapsed_time:208.52232


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch17/20, loss:0.06383, acc:0.73432, val_loss:0.07816, val_acc:0.69801, elapsed_time:209.01418


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch18/20, loss:0.06494, acc:0.72590, val_loss:0.08132, val_acc:0.69419, elapsed_time:207.35563


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch19/20, loss:0.06282, acc:0.73814, val_loss:0.07752, val_acc:0.68807, elapsed_time:207.70525


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch20/20, loss:0.06234, acc:0.73432, val_loss:0.07799, val_acc:0.69801, elapsed_time:207.97018


In [25]:
# Unfreeze the layer 3 and do training again
for param in model.layer3.parameters():
    param.requires_grad = True
optimizer = AdamW(model.parameters(), lr=tc.lr)

In [26]:
# and do training again
for epoch in range(tc.epoch_count):
    # time one cycle of train and validate
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, validation_loader)
    elapsed_time = time.time() - start_time
    
    # save the weight when the loss gets smaller than last one
    if val_loss < best_val_loss:
        save_model(model, tc.device)
        print("WEIGHTS-ARE-SAVED")
        best_val_loss = val_loss
        
    print(f"Epoch{epoch+1}/{tc.epoch_count}, loss:{train_loss:.5f}, acc:{train_acc:.5f}, val_loss:{val_loss:.5f}, val_acc:{val_acc:.5f}, elapsed_time:{elapsed_time:.5f}")
    # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
    item = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc, elapsed_time])
    # store history for matplotlib visualization
    history = np.vstack((history, item))

    # For tensorboard
    tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_loss, epoch+1)
    tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_acc, epoch+1)
    tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)

  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch1/20, loss:0.06126, acc:0.74005, val_loss:0.07901, val_acc:0.69801, elapsed_time:217.09165


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch2/20, loss:0.05950, acc:0.74464, val_loss:0.07675, val_acc:0.70260, elapsed_time:214.73384


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch3/20, loss:0.05747, acc:0.75574, val_loss:0.07992, val_acc:0.70642, elapsed_time:213.87156


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch4/20, loss:0.05722, acc:0.75995, val_loss:0.07854, val_acc:0.70183, elapsed_time:215.05072


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch5/20, loss:0.05428, acc:0.77697, val_loss:0.07414, val_acc:0.71330, elapsed_time:215.82531


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch6/20, loss:0.05272, acc:0.77888, val_loss:0.07731, val_acc:0.70948, elapsed_time:215.23166


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch7/20, loss:0.05128, acc:0.78405, val_loss:0.07729, val_acc:0.70948, elapsed_time:219.11987


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch8/20, loss:0.05079, acc:0.78864, val_loss:0.07574, val_acc:0.69725, elapsed_time:219.54557


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch9/20, loss:0.05179, acc:0.79151, val_loss:0.07280, val_acc:0.72554, elapsed_time:217.65029


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch10/20, loss:0.04985, acc:0.79017, val_loss:0.07675, val_acc:0.69419, elapsed_time:216.61193


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch11/20, loss:0.04874, acc:0.79208, val_loss:0.07658, val_acc:0.71101, elapsed_time:217.92556


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch12/20, loss:0.04874, acc:0.79629, val_loss:0.07354, val_acc:0.71560, elapsed_time:217.19097


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch13/20, loss:0.04680, acc:0.80604, val_loss:0.07217, val_acc:0.73242, elapsed_time:217.24203


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch14/20, loss:0.04784, acc:0.79992, val_loss:0.07329, val_acc:0.73012, elapsed_time:219.32309


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch15/20, loss:0.04504, acc:0.81695, val_loss:0.07253, val_acc:0.73853, elapsed_time:217.68648


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch16/20, loss:0.04775, acc:0.80547, val_loss:0.07273, val_acc:0.73700, elapsed_time:218.72592


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch17/20, loss:0.04526, acc:0.81197, val_loss:0.07317, val_acc:0.72554, elapsed_time:218.95729


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch18/20, loss:0.04596, acc:0.81331, val_loss:0.07366, val_acc:0.72095, elapsed_time:217.66716


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch19/20, loss:0.04456, acc:0.82116, val_loss:0.07331, val_acc:0.72936, elapsed_time:214.03841


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch20/20, loss:0.04250, acc:0.82173, val_loss:0.07376, val_acc:0.72401, elapsed_time:215.40622


In [27]:
# Unfreeze ALL backborn and do training again
for param in model.parameters():
    param.requires_grad = True
optimizer = AdamW(model.parameters(), lr=tc.lr)

In [28]:
# and do training again
for epoch in range(tc.epoch_count):
    # time one cycle of train and validate
    start_time = time.time()
    train_loss, train_acc = train(model, train_loader, optimizer)
    val_loss, val_acc = validate(model, validation_loader)
    elapsed_time = time.time() - start_time
    
    # save the weight when the loss gets smaller than last one
    if val_loss < best_val_loss:
        save_model(model, tc.device)
        print("WEIGHTS-ARE-SAVED")
        best_val_loss = val_loss
        
    print(f"Epoch{epoch+1}/{tc.epoch_count}, loss:{train_loss:.5f}, acc:{train_acc:.5f}, val_loss:{val_loss:.5f}, val_acc:{val_acc:.5f}, elapsed_time:{elapsed_time:.5f}")
    # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
    item = np.array([epoch+1, train_loss, train_acc, val_loss, val_acc, elapsed_time])
    # store history for matplotlib visualization
    history = np.vstack((history, item))

    # For tensorboard
    tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_loss, epoch+1)
    tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count), train_acc, epoch+1)
    tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch1/20, loss:0.04241, acc:0.82536, val_loss:0.07411, val_acc:0.71942, elapsed_time:238.95527


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch2/20, loss:0.04070, acc:0.83359, val_loss:0.07296, val_acc:0.72783, elapsed_time:236.66323


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch3/20, loss:0.03911, acc:0.83646, val_loss:0.07348, val_acc:0.71560, elapsed_time:237.72076


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch4/20, loss:0.03919, acc:0.83703, val_loss:0.07385, val_acc:0.71942, elapsed_time:240.16036


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch5/20, loss:0.03813, acc:0.84449, val_loss:0.07194, val_acc:0.72630, elapsed_time:237.36145


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch6/20, loss:0.03980, acc:0.83875, val_loss:0.07387, val_acc:0.73012, elapsed_time:236.45559


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch7/20, loss:0.03723, acc:0.85291, val_loss:0.07226, val_acc:0.73394, elapsed_time:236.53716


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch8/20, loss:0.03749, acc:0.84621, val_loss:0.07382, val_acc:0.73394, elapsed_time:235.26438


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch9/20, loss:0.03368, acc:0.86477, val_loss:0.07261, val_acc:0.73089, elapsed_time:238.46773


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

WEIGHTS-ARE-SAVED
Epoch10/20, loss:0.03313, acc:0.86266, val_loss:0.07162, val_acc:0.71942, elapsed_time:236.34955


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch11/20, loss:0.03389, acc:0.86362, val_loss:0.07451, val_acc:0.73089, elapsed_time:236.51609


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch12/20, loss:0.03289, acc:0.86744, val_loss:0.07535, val_acc:0.72171, elapsed_time:237.29888


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch13/20, loss:0.03365, acc:0.86572, val_loss:0.07562, val_acc:0.71789, elapsed_time:237.76888


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch14/20, loss:0.03269, acc:0.86515, val_loss:0.07502, val_acc:0.73012, elapsed_time:238.71263


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch15/20, loss:0.03161, acc:0.86859, val_loss:0.07402, val_acc:0.72171, elapsed_time:234.55532


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch16/20, loss:0.03135, acc:0.87108, val_loss:0.07303, val_acc:0.72783, elapsed_time:235.72098


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch17/20, loss:0.03192, acc:0.86974, val_loss:0.07298, val_acc:0.73012, elapsed_time:233.37622


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch18/20, loss:0.03072, acc:0.86744, val_loss:0.07466, val_acc:0.72630, elapsed_time:235.32455


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch19/20, loss:0.02861, acc:0.88466, val_loss:0.07571, val_acc:0.71942, elapsed_time:238.12016


  0%|          | 0/403 [00:00<?, ?it/s]

  0%|          | 0/101 [00:00<?, ?it/s]

Epoch20/20, loss:0.02805, acc:0.88982, val_loss:0.07448, val_acc:0.72706, elapsed_time:240.93692


In [29]:
# Now do the inference and get submission.csv file
test_dataset = KenyanFood13Testset()
test_loader = DataLoader(test_dataset, batch_size = tc.batch_size, shuffle = False)

# prediction
predicted_result = predict(model, test_loader)
#print("predicted_result len:{}".format(len(predicted_result)))
classifications = []

#change predicted index to class_names
for index in range(len(predicted_result)):
    predicted_result_class = test_dataset.base_dataset.index2class(int(predicted_result[index]))
    classifications.append(predicted_result_class)

# put it to csv file
classes = pd.DataFrame(classifications, columns = ["class"])
result = test_dataset.test_labels.join(classes)
result.to_csv(tc.submission_csv, index=False, header = True)

In [30]:
# dont use anymore... refactored
# def fit(model, optimizer, train_loader, validation_loader, history):
#     """works like tensorflow fits. epoch_count, optimizer and criterion(e.g., CrossEntorpyLoss) from training config"""
    
#     #from tqdm.notebook import tqdm
    
#     base_epochs = len(history)
    
#     for epoch in range(base_epochs, tc.epoch_count + base_epochs):
#         train_loss = 0
#         train_acc = 0
#         val_loss = 0
#         val_acc = 0
#         best_valid_loss = np.Inf
        
#         # training Phase
#         model.train()
#         count = 0
        
#         t_begin = time.time()
#         for inputs, labels in train_loader:
            
#             count += len(labels)
#             inputs = inputs.to(tc.device)
#             labels = labels.to(tc.device)
            
#             # initialize the gradients
#             optimizer.zero_grad()
#             # predict
#             outputs = model(inputs)
#             # calculate loss
#             loss = tc.criterion(outputs, labels)
#             train_loss += loss.item()
            
#             #calucalte the graidents
#             loss.backward()
#             #update parameter
#             optimizer.step()
            
#             # get highest predicted value
#             predicted = torch.max(outputs, axis=1)[1]
#             # calculate number of correct answer
#             train_acc  += (predicted == labels).sum().item()
            
#             #calculate the average loss and accuracy in the batch
#             avg_train_loss = train_loss / count
#             avg_train_acc = train_acc / count
            
#         #evaluation phase
#         model.eval()
#         count = 0
        
#         for inputs, labels in validation_loader:
#             count += len(labels)
#             inputs = inputs.to(tc.device)
#             labels = labels.to(tc.device)
            
#             # calculate the prediction
#             outputs = model(inputs)
            
#             # calculate the loss
#             loss = tc.criterion(outputs, labels)
#             val_loss += loss.item()
            
#             if val_loss < best_valid_loss:
#                 torch.save(model.state_dict(), "best_model.pt")
#                 print("WEIGHTS-ARE-SAVED")
#                 best_valid_loss = val_loss
                
#             # get highest predicted value
#             predicted = torch.max(outputs, axis=1)[1]
#             # calculate number of correct answer
#             val_acc += (predicted == labels).sum().item()
            
#             #calculate the average loss and accuracy in the batch
#             avg_val_loss = val_loss / count
#             avg_val_acc = val_acc / count
            
            
#         elapsed_time = time.time() - t_begin
#         # print out
#         print("Epoch{}/{},  loss:{:.5f}, acc:{:.5f}, val_loss:{:.5f}, val_acc:{:.5f}, elapsed_time:{:.5f}".format(
#             epoch+1,tc.epoch_count+base_epochs, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, elapsed_time
#         ))
#         # store 0:epoch, 1:avg_train_loss, 2: avg_train_acc, 3: avg_val_loss, 4: avg_val_acc, 5: elapsed_time
#         item = np.array([epoch+1, avg_train_loss, avg_train_acc, avg_val_loss, avg_val_acc, elapsed_time])
#         history = np.vstack((history, item))
        
#         # For tensorboard
#         tc.tb_writer.add_scalar('Loss/Train within {}/{}'.format(epoch+1,tc.epoch_count+base_epochs), avg_train_loss, epoch+1)
#         tc.tb_writer.add_scalar('Accuracy/Train within {}/{}'.format(epoch+1,tc.epoch_count+base_epochs), avg_train_acc, epoch+1)
#         tc.tb_writer.add_scalar('Time/elapsed_time', elapsed_time, epoch+1)

        
#     return history

## <font style="color:green">8. TensorBoard Dev Scalars Log Link [5 Points]</font>

**Share your TensorBoard scalars logs link here You can also share (not mandatory) your GitHub link, if you have pushed this project in GitHub.**


For example, [Find Project2 logs here](https://tensorboard.dev/experiment/kMJ4YU0wSNG0IkjrluQ5Dg/#scalars).

/kaggle/working/logs/events.out.tfevents.1676751468.7b40718cd348.24.0


In [31]:
https://github.com/caq05630

SyntaxError: invalid syntax (2905022292.py, line 1)

## <font style="color:green">9. Kaggle Profile Link [50 Points]</font>

**Share your Kaggle profile link  with us here to score , points in  the competition.**

**For full points, you need a minimum accuracy of `75%` on the test data. If accuracy is less than `70%`, you gain  no points for this section.**


**Submit `submission.csv` (prediction for images in `test.csv`), in the `Submit Predictions` tab in Kaggle, to get evaluated for  this section.**

https://www.kaggle.com/code/yasuokada/project2/edit/run/119205728